# Imports

In [1]:
import os
import re
import math
import requests
import numpy as np
import pandas as pd
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from datetime import datetime

## Get infos for one city - Düsseldorf

In [7]:
# only wohnung

url = 'https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city=100207&marketingtype=2&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, "html.parser")

# Find number of rent offers.
a = soup.findAll('h1', class_='box-50')[0].get_text()
total_offers = int(re.search('\d+', a).group())

## Get all offers ids for Düsseldorf - only Wohnung (apartments)

In [493]:
offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

# Get the number of pages to scrape - rounded to down
number_of_pages = math.floor(total_offers / len(offers_list_1page))

# City code
city_code = "100207" # Düsseldorf

# List of all ids
ids_list = []


for page in range(number_of_pages):
    
    url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

    for i in range(len(offers_list_1page)):
        try:
            ids_list.append(offers_list_1page[i].find('a')['data-object-id'])
        except:
            pass
        #sleep(randint(1, 4))

In [494]:
len(ids_list)

366

In [495]:
len(set(ids_list))

341

## get infos for each offer_id

In [2]:
'''
Here we will get the infos from the info panels for each offer.
'''
# City code
city_code = "100207"
city = 'Dusseldorf'

infos_list = []
now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

for Id in set(ids_list):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
    
    infos_list.append({'offer_id': Id, 'datetime': now, 'city': city, 'city_code': city_code, 'infos': offer_infos})
    
    if len(offer_infos) == 0:
        print(Id, ' - ERROR')    

NameError: name 'ids_list' is not defined

In [498]:
df_raw = pd.DataFrame(infos_list)

In [499]:
df_raw.head()

,offer_id,datetime,infos
0,45383090,20211003-191642,[-----WOHNEN UNKOMPLIZIERT - EIN PREIS ALLES D...
1,45155366,20211003-191642,[-----Helle 2 Zimmer Wohnung mit Dachterrasse-...
2,45013603,20211003-191642,[-----Voll möbliertes Apartment mit hotelähnli...
3,45299018,20211003-191642,"[-----Düsseldorf-Bilk! Ruhige Seitenstraße, 2 ..."
4,45382436,20211003-191642,"[-----Helle 3-Zimmerwohnung mit Balkon, Wannen..."


In [481]:
df_raw['infos'][0]

['-----Unterrath: Helles Appartement am Kartäuserpark mit Einbauküche, modernisiertem Tageslichtbad und Balkon----------------Apartment ------mieten-------------------in Düsseldorf-----------------Kartäuserstrasse   . .------------40468\xa0------Düsseldorf------------Auf Karte anzeigenDieses Angebot teilenDieses Angebot teilen',
 'Meine Orte',
 'Preise & Kosten----------Miete zzgl. NK----------525.0\xa0€---------------------Miete inkl. NK----------625.0\xa0€---------------------Nebenkosten----------100.0\xa0€-----------Heizkosten in Nebenkosten enthalten:Ja----------Kaution/ Genossenschaftsanteile----------1575.0\xa0€-----------',
 'Größe & Zustand-----------------------Zimmer--------------------1.0------Wohnfläche ca.------40.0\xa0m²Zustand-------Teil- oder vollsaniert------Baujahr1968Verfügbar ab--------------sofort-------------',
 'EnergieEnergieeffizienzklasseKlasse F----------------------------Endenergieverbrauch--------------------------188.0 kWh/(m²*a)--------Energieausweis-----

## Get offers for houses too

In [22]:
# get all offers quantity - haus und wohnung

total_offers = 0

for i in range(1,3):
    url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city=100207&marketingtype=2&page=1'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Find number of rent offers.
    a = soup.findAll('h1', class_='box-50')[0].get_text()
    total_offers += int(re.search('\d+', a).group())

In [23]:
# get all offers ids for haus und wohnung in Dusseldorf

offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

# Get the number of pages to scrape - rounded to down
number_of_pages = math.floor(total_offers / len(offers_list_1page))

# City code
city_code = "100207" # Düsseldorf
city = 'Dusseldorf'

# List of all ids
ids_list = []

# wohnung/haus code
l_opt = [1, 2]

for opt in l_opt:
    for page in range(number_of_pages):
        url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat={opt}&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
    
        offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

        for i in range(len(offers_list_1page)):
            try:
                if opt == 1:
                    ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id']'city': city, 'city_code': city_code, 'type': 'wohnung'})
                if opt == 2:
                    ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'haus'})
            except:
                pass
            
# Create a dataframe with the infos
df_ids = pd.DataFrame(ids_list)

In [29]:
'''
Here we will get the infos from the info panels for each offer.
'''

infos_list = []
now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

for Id in set(df_ids['offer_id']):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
    
    infos_list.append({'offer_id': Id, 'datetime': now, 'city': city, 'city_code': city_code, 'infos': offer_infos})
    
    if len(offer_infos) == 0:
        print(Id, ' - ERROR')   
        
# create a dataframe
df_raw_infos = pd.DataFrame(infos_list)

In [30]:
df_raw_infos = pd.DataFrame(infos_list)

In [32]:
df_raw_infos.shape

(438, 5)

In [3]:
# get main German cities codes

cities_dict = {
    'Dusseldorf': 100207,
    'Berlin': 87372,
    'Essen': 102157,
    'Munchen': 121673,
    'Koln': 113144,
    'Stuttgart': 143262,
    'Dresden': 100051,
    'Hannover': 109489,
    'Dortmund': 99990,
    'Frankfurt am Main': 105043,
    'Hamburg': 109447
}

# get all offers quantity - haus und wohnung

cities_offers = []

for city, code in cities_dict.items():
    
    total_offers = 0
    
    for i in range(1,3):
        url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city={code}&marketingtype=2&page=1'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Find number of rent offers.
        a = soup.findAll('h1', class_='box-50')[0].get_text()
        total_offers += int(re.search('\d+', a).group())
    
    cities_offers.append({'city': city, 'city_code': code, 'offers': total_offers})
    
offers_by_page = len(soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter"))    
df_offers = pd.DataFrame(cities_offers)

In [12]:
# List of all ids
ids_list = []
offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

for c in range(len(df_offers)):
    city = df_offers.loc[c]['city']
    city_code = df_offers.loc[c]['city_code']
    offers = df_offers.loc[c]['offers']
    
    # get all offers ids for haus und wohnung in each city
    

    # Get the number of pages to scrape - rounded to down
    number_of_pages = math.floor(offers / offers_by_page)
    
    # wohnung/haus code
    l_opt = [1, 2]

    for opt in l_opt:
        for page in range(number_of_pages):
            url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat={opt}&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, "html.parser")

            offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

            for i in range(len(offers_list_1page)):
                try:
                    if opt == 1:
                        ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'wohnung'})
                    if opt == 2:
                        ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'haus'})
                except:
                    pass
            sleep(randint(1, 2)) 
    sleep(randint(1, 5))          
            
# Create a dataframe with the infos
df_ids = pd.DataFrame(ids_list)

# save as csv file
now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../data/'
filename = f'immonet_de_{now2}.csv'
df_ids.to_csv(os.path.join(output_dir, filename), index=False)

In [5]:
# get all infor for all founded offers

'''
Here we will get the infos from the info panels for each offer.
'''

infos_list = []
datetime_list = []
# now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

for Id in set(df_ids['offer_id']):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
    
    '''
    infos_list.append({'offer_id': Id,
                       'datetime': now,
                       #'city': city,
                       #'city_code': city_code,
                       'infos': offer_infos})
    ''''
    datetime_list.append(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))
    infos_list.append(offer_infos)
    #if len(offer_infos) == 0:
        #print(Id, ' - ERROR') 

#df_offers_infos = pd.DataFrame(infos_list)
df_ids['scrapy_time'] = datetime_list
df_ids['infos'] = infos_list

45514797  - ERROR
45613134  - ERROR
45613502  - ERROR
45580771  - ERROR
45548311  - ERROR
45417347  - ERROR
45286402  - ERROR
45549453  - ERROR
45549787  - ERROR
45583007  - ERROR
45550286  - ERROR
45583624  - ERROR
45551407  - ERROR
45551660  - ERROR
45551741  - ERROR
45552020  - ERROR
45552108  - ERROR
45127051  - ERROR
45585908  - ERROR
45389579  - ERROR
45553444  - ERROR
45520771  - ERROR
34707844  - ERROR
34708006  - ERROR
45489311  - ERROR
45358645  - ERROR
45490063  - ERROR
45555708  - ERROR
45457530  - ERROR
45490338  - ERROR
45491088  - ERROR
45589470  - ERROR
45557280  - ERROR
45491949  - ERROR
45492133  - ERROR
45492401  - ERROR
45492404  - ERROR
45492555  - ERROR
45492597  - ERROR
45361946  - ERROR
45296617  - ERROR
45493398  - ERROR
45591922  - ERROR
45592048  - ERROR
45264583  - ERROR
45560033  - ERROR
45593501  - ERROR
45528028  - ERROR
45397097  - ERROR
45364456  - ERROR
45528843  - ERROR
45496120  - ERROR
45430623  - ERROR
45529179  - ERROR
34683783  - ERROR
45366394  

In [20]:
df_ids.head(1)

,offer_id,city,city_code,type
0,45610889,Dusseldorf,100207,wohnung


In [21]:
df_offers_infos.head(1)

,offer_id,datetime,infos
0,45514753,2021-10-10-10:54:54,['-----Exklusive teilmöblierte 3-Zi.-Wohnung m...


In [8]:
df_ids = pd.read_csv('../data/immonet_de_2021_10_09_13_26_04.csv')
df_offers_infos = pd.read_csv('../data/all_offers_infos.csv')

In [34]:
# join both dataframes and save it

df_full = pd.concat([df_ids, df_offers_infos], join='inner', axis=1, ignore_index=True)

In [36]:
df_full.head()

,0,1,2,3,4,5,6
0,45610889,Dusseldorf,100207,wohnung,45514753,2021-10-10-10:54:54,['-----Exklusive teilmöblierte 3-Zi.-Wohnung m...
1,45578967,Dusseldorf,100207,wohnung,45580290,2021-10-10-10:54:54,"['-----3,0-Zimmer-Wohnung in Hannover Sahlkamp..."
2,45475784,Dusseldorf,100207,wohnung,44924940,2021-10-10-10:54:54,['-----Frisch sanierte 2 Raum Wohnung in ruhig...
3,45466649,Dusseldorf,100207,wohnung,44630030,2021-10-10-10:54:54,['-----wunderschön möbilierte Wohnung gegenübe...
4,45558112,Dusseldorf,100207,wohnung,45514780,2021-10-10-10:54:54,['-----Erstaunlich gut wohnen! VIVAWEST lässt ...


In [35]:
df_full.rename(columns=[])

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [ ]:
df_full.to_csv('../data/all_offers_infos_full.csv', index=False)

## Script

In [ ]:
import os
import re
import math
import requests
import numpy as np
import pandas as pd
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from datetime import datetime

'''Extract information from all rent offers from immonet.de

This scrip gets all the informations for each rent offer (house or appartment)
from immonet.de website for the main German cities and save either as a csv file 
or in a postgres database.

The extracted information is saved as str in a raw format in the column "infos".

'''

def get_offers_by_city():
    '''Returns the total number of rent offers (houses and appartments) for each city.
    
    '''
    
    # main German cities codes.
    cities_dict = {
        'Dusseldorf': 100207,
        'Berlin': 87372,
        'Essen': 102157,
        'Munchen': 121673,
        'Koln': 113144,
        'Stuttgart': 143262,
        'Dresden': 100051,
        'Hannover': 109489,
        'Dortmund': 99990,
        'Frankfurt am Main': 105043,
        'Hamburg': 109447
    }

    # list with all offers quantity - haus und wohnung - by city.
    cities_offers = []
    
    # get the number of offers for each city.
    for city, code in cities_dict.items():
        total_offers = 0

        for i in range(1,3):
            url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city={code}&marketingtype=2&page=1'
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, "html.parser")

            # Find number of rent offers.
            a = soup.findAll('h1', class_='box-50')[0].get_text()
            total_offers += int(re.search('\d+', a).group())
        
        # append the offers in a dict format in cities_offers list
        cities_offers.append({'city': city, 'city_code': code, 'offers': total_offers})

    #offers_by_page = len(soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")) 
    
    df_offers = pd.DataFrame(cities_offers)
    
    return df_offers

def get_offers_ids(df_offers):
    '''Gets all the rent offers ids for each city
    '''
    
    # List of all ids
    ids_list = []
    offers_by_page = 26
    offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

    for c in range(len(df_offers)):
        city = df_offers.loc[c]['city'] 
        city_code = df_offers.loc[c]['city_code']
        offers = df_offers.loc[c]['offers']

        # Get the number of pages to scrape for each city
        number_of_pages = math.floor(offers / offers_by_page)

        # wohnung/haus code
        l_opt = [1, 2]

        for opt in l_opt: # switch into appartments and houses
            for page in range(number_of_pages):
                url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat={opt}&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
                headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
                page = requests.get(url, headers=headers)
                soup = BeautifulSoup(page.text, "html.parser")
                
                # all the offers for 01 page
                offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")
                
                # for loop to get all the offers for all pages for all cities
                for i in range(len(offers_list_1page)):
                    try:
                        if opt == 1:
                            ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'wohnung'})
                        if opt == 2:
                            ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'haus'})
                    except:
                        pass
                    
                # Some wait time to not overload the website.    
                sleep(randint(1, 2)) 
        sleep(randint(1, 5))          

    # Create a dataframe with the infos
    df_ids = pd.DataFrame(ids_list)
    
    return df_ids

def get_offers_infos(df_ids, save=False):
    '''Get all the infos from each rent offer and save it in a csv file. 
    
    The infos comes in row format.
    
    Parameters
    ----------
    df_ids: DataFrame
        DataFrame with all the offers ids to be scraped
    
    save: boolean
        if the infos will be saved or not as a csv file.
    '''
    
    # create a list to store all the information about each offer
    infos_list = []
    
    # scrape time
    now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

    for Id in set(df_ids['offer_id']):

        url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")

        panels_infos = soup.findAll('div', class_='row box-50')
        
        # get all infos about each offer (all infos are mixed)
        offer_infos = []
        for panel in panels_infos:
            offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
        
        # store the infos for each offer in a list inside a list
        infos_list.append({'offer_id': Id,
                           'datetime': now,
                           #'city': city,
                           #'city_code': city_code,
                           'infos': offer_infos})
        
        # print an error when a offer is no longuer available or returned no information
        if len(offer_infos) == 0:
            print(Id, ' - ERROR') 

    df_offers_infos = pd.DataFrame(infos_list)
    
    print(f'Was extracted {df_offers_infos.shape[0]} offers.')
    
    if save:
        # save as csv file
        now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        output_dir = '../data/'
        filename = f'immonet_de_infos_{now2}.csv'
        df_ids.to_csv(os.path.join(output_dir, filename), index=False)
    
    return df_offers_infos


def postgres_storage(df_offers_infos):
    '''
    Saves the row_infos.csv in a postgres database
    '''